# Evolutionary transition rates of leaf endopolyploidy

Written by James Seery (jseery@mail.uoguelph.ca)

## Setting up

#### Load packages and functions

In [ ]:
tryCatch(    # Load corHMM package. Provides the functions for ancestral reconstruction and inference of transition rates.
    library(corHMM),
    error = install.packages("corHMM", repos = "http://cran.utstat.utoronto.ca/")
)
tryCatch(    # Load CAPER package. Provides the ability to prune a large phylogeny.
    library(caper),
    error = install.packages("caper", repos = "http://cran.utstat.utoronto.ca/")
)
source(corHMM_functions.R)

#### Import dataset: entire Zanne phylogeny (phylo); and species data (endo)

In [ ]:
phylo = read.tree("Zanne.nwk")
phylo = makeLabels(phylo) # Add unique labels to all unnamed nodes
endo = read.csv("EI.csv") # Species names that match the tree in column 1 (labeled 'name', and leaf endoreduplication index in column 2 (labeled 'state').

#### Turn endopolyploidy into a character with two states

In [ ]:
threshold = 0.1 # Specify threshold endoreduplication index for deciding whether a species has endopolyploid leaves.
endo$state = state_threshold(endo$state, threshold)

#### Match the phylogeny (~30,000 species) to the dataset (~168 species)

In [ ]:
data = comparative.data(phy = phylo,
                        data = endo,
                        names.col = "name")

## Modelling evolutionary transition rates of leaf endopolyploidy

#### Asymmetric transition models

In [ ]:
Asym_ancestorNonEndo = transition_model(data, 
                                       model = "ARD", 
                                       root_nonendo = TRUE)
Asym_ancestorUnknown = transition_model(data, 
                                       model = "ARD")
Aikake_weight(Asym_ancestorNonEndo, Asym_ancestorUnknown) # If assuming ancestor is equal is significantly better, choose Asym_ancestorUnknown. Otherwise choose Asym_ancestorNonEndo

#### Symmetric transition models

In [ ]:
Sym_ancestorNonEndo = transition_model(data, 
                                       model = "ER", 
                                       root_nonendo = TRUE)
Sym_ancestorUnknown = transition_model(data, 
                                       model = "ER")
Aikake_weight(Sym_ancestorNonEndo, Sym_ancestorUnknown) # If assuming ancestor is equal is significantly better, choose Sym_ancestorUnknown. Otherwise choose Sym_ancestorNonEndo

#### Is the asymmetric model better?

In [ ]:
AIC_ancestorNonEndo = Aikake_weight(Asym_ancestorNonEndo, Sym_ancestorNonEndo) # Null hypothesis is symmetric transitions.
AIC_ancestorUnknown = Aikake_weight(Asym_ancestorUnknown, Sym_ancestorUnknown)

if(AIC_ancestorNonEndo[1] > 0.95 && AIC_ancestorUnknown[1] > 0.95) print("The best model is asymmetric transitions")
else print("The best model may have symmetric transitions") # Refine this conditional.

## Modelling transitions with woodiness

Endopolyploid leaves and woodiness do not co-occur. Redesign models to re-run the above code including woodiness.

In [ ]:
# . . . 

transition_matrix = rate.mat.maker(hrm=FALSE,ntraits=1,nstates=3,model="ARD") # If the asymmetric model is best, otherwise use model="ER".